## Decorators
Fiunctions are first-class objects: they be bound to names, passed as arguments to other functions, be returned functions.

In [134]:
def greet(name):
    return "hello " + name
groet = greet
groet("Albert")   

'hello Albert'

In [141]:
def greeting(name):
    def message():               # functions can be defined in (by) other functions 
        return f"Hello {name}"   # note that 'name' is accessible in embedded function
    return message()
greeting('Joe')

'Hello Joe'

In [135]:
# Function can return another function:
def greet(name):
    def message():               
        return f"Hello {name}"   
    return message       # return function instead of just its result
hi_joe = greet('Joe')
hi_joe()

'Hello Joe'

In [139]:
#Function can take other function as an argument:
def call_func(func, *args):
    return func(*args)  
call_func(hi_joe), call_func(greet('Joe'))

('Hello Joe', 'Hello Joe')

### Closure
Free variables in a function that were bound at definition time remain so when the function is later executed. See the function returned by `greet` above. Below is another example: the b remains bound to 4. This example also shows that functions can, like other Python objects, be given extra attributes, both from inside and from outside the function body. This is different from a closure: foo can produce many different closures for baz 

In [2]:
a = 10
def foo(a):
    foo.a += 9
    b = 4
    def baz():
        print(a, b, foo.a)
    baz.x = 7
    return baz

foo.a = 8
print(foo.__dict__)
foo(5)()
print(foo.__dict__)
foo.a = 3
foo(7)()
foo(8).x
foo(8).x = 132
foo(8).x

{'a': 8}
5 4 17
{'a': 17}
7 4 12


7

Existing functions can be decorated with additional functionality without changes the function itself, by wrapping the function in another function and naming the resulting wrapped function the same as the function being wrapped:

In [15]:
def wrapper(f):
    def wrapped(*args, **kwargs):
        print("A wrapped version of", f.__name__)
        return f(*args, **kwargs)
    return wrapped

def foo(*args):
    return sum(args)

foo = wrapper(foo)
foo(*range(9))

A wrapped version of foo


36

Syntactic shortcut is provided by @ keyword:

In [27]:
@wrapper
def baz(*args, ignore=3):
    "sum ignoring multiples of three"
    return sum(filter(lambda x: x% ignore, args))

baz(*range(14))

A wrapped version of baz


61

In [28]:
help(baz)
baz.__name__

Help on function wrapped in module __main__:

wrapped(*args, **kwargs)



'wrapped'

Not very useful. Need to move __name__, __doc__ and from wrapped to wrapper. 

In [34]:
def wrapper(f):
    def wrapped(*args, **kwargs):
        print("A wrapped version of", f.__name__)
        return f(*args, **kwargs)
    wrapped.__doc__ = f.__doc__
    wrapped.__qualname__ = f.__qualname__
    return wrapped
@wrapper
def baz(*args, ignore=3):
    "sum ignoring multiples of three"
    return sum(filter(lambda x: x% ignore, args))
help(baz)
print(baz)

Help on function wrapped in module __main__:

wrapped(*args, **kwargs)
    sum ignoring multiples of three

<function baz at 0x000001D41B773048>


In [36]:
# easier (and better: also copies signature!) using functools.wraps decorator
from functools import wraps
def wrapper(f):
    @wraps(f)
    def wrapped(*args, **kwargs):
        print("A wrapped version of", f.__name__)
        return f(*args, **kwargs)
    return wrapped
@wrapper
def baz(*args, ignore=3):
    "sum ignoring multiples of three"
    return sum(filter(lambda x: x% ignore, args))
help(baz)
print(baz)

Help on function baz in module __main__:

baz(*args, ignore=3)
    sum ignoring multiples of three

<function baz at 0x000001D41B773288>


In [37]:
#stacking decorators
def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper
def strong_decorate(func):
    def func_wrapper(name):
        return "<strong>{0}</strong>".format(func(name))
    return func_wrapper
def div_decorate(func):
    def func_wrapper(name):
        return "<div>{0}</div>".format(func(name))
    return func_wrapper
@div_decorate
@p_decorate
@strong_decorate
def get_text(name):
    return "Hi {0}!".format(name)
get_text("Mary")

'<div><p><strong>Hi Mary!</strong></p></div>'

In [43]:
# decorator with args
from functools import wraps
def debug(prefix=''):
    def decorator(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            print(f'{prefix} {func.__qualname__}')
            return func(*args, **kwargs)
        return wrapped
    return decorator
@debug(prefix='###')
def foo(x):
    return x/2
foo(9)

### foo


4.5

In [45]:
# far too clever: use decorator twice, using partial to create closure, but requiring named 'prefix' 
from functools import wraps, partial 
def debug(func=None, *, prefix= ''):
    if func is None:
        return partial(debug, prefix=prefix)
    @wraps(func)
    def wrapped(*args, **kwargs):
        print(f'{prefix} {func.__qualname__}')
        return func(*args, **kwargs)
    return wrapped   
@debug(prefix='###')
def foo(x):
    return x/2
foo(9)

### foo


4.5

In [46]:
# class decorator
instances = {}
def singleton(cls):
    def wrapped(*args):
        if cls not in instances:
             instances[cls]=cls(*args)
        return instances[cls]
    return wrapped
@singleton
class Foo: pass
Foo() is Foo()

True

In [61]:
# class decorato: store instances with decorator
def singleton(cls):
    singleton.instances = {}
    def wrapped(*args):
        if cls not in singleton.instances:
             singleton.instances[cls]=cls(*args)
        return singleton.instances[cls]
    return wrapped
@singleton
class Foo: pass
@singleton
class Baz: pass
Foo() is Foo(), Baz() is Baz(), singleton.instances

(True,
 True,
 {__main__.Foo: <__main__.Foo at 0x1d41aa92148>,
  __main__.Baz: <__main__.Baz at 0x1d41aa921c8>})

In [65]:
def singleton(cls):
    instances = {}
    def wrapped(*args):
        if cls not in instances:
             instances[cls]=cls(*args)
        return instances[cls]
    return wrapped
@singleton
class Foo: pass
@singleton
class Baz: pass
Foo() is Foo(), Baz() is Baz() # hard or impossible (?) to get hold of dictionary

(True, True)

In [58]:
# class decorator
def singleton(cls):
    instance = None
    def wrapped(*args):
        nonlocal instance
        if not instance:
             instance=cls(*args)
        return instance
    return wrapped
@singleton
class Foo: pass
@singleton
class Baz: pass
a = Baz()
f = Foo()
a is Baz(), f is Foo()

(True, True)

### Callables with state: three options

In [66]:
#Closure

def adder_as_closure(augend):
    def add(addend, _augend=augend):
        return addend+_augend
    return add
adder_as_closure(5)(4)

9

In [67]:
#Bound method

def adder_as_bound_method(augend):
    class Adder:
        def __init__(self, augend):
            self.augend = augend
        def add(self, addend):
            return addend+self.augend   
    return Adder(augend).add
adder_as_bound_method(5)(4)

9

In [68]:
#Callable instance

def adder_as_callable_instance(augend):
    class Adder:
        def __init__(self, augend):
            self.augend = augend
        def __call__(self, addend):
            return addend+self.augend
    return Adder(augend)
adder_as_callable_instance(5)(4)

9

In [75]:
list(add(2) for add in (make(13) for make in [adder_as_closure, adder_as_bound_method, adder_as_callable_instance]))

[15, 15, 15]